In [20]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib
from statsmodels.tsa.stattools import grangercausalitytests
from pylab import plt, mpl

In [3]:
df = pd.read_csv('aiif_eikon_eod_data.csv').set_index('Date').dropna()
df.head()

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,SPY,.SPX,.VIX,EUR=,XAU=,GDX,GLD
Date,,,,,,,,,,,,
2010-01-04,30.572827,30.950,20.88,133.90,173.08,113.33,1132.99,20.04,1.4411,1120.00,47.71,109.80
2010-01-05,30.625684,30.960,20.87,134.69,176.14,113.63,1136.52,19.35,1.4368,1118.65,48.17,109.70
2010-01-06,30.138541,30.770,20.80,132.25,174.26,113.71,1137.14,19.16,1.4412,1138.50,49.34,111.51
2010-01-07,30.082827,30.452,20.60,130.00,177.67,114.19,1141.69,19.06,1.4318,1131.90,49.10,110.82
2010-01-08,30.282827,30.660,20.83,133.52,174.31,114.57,1144.98,18.13,1.4412,1136.10,49.84,111.37


In [4]:
df_stock = df.iloc[:,:4]
df_stock.head()

,AAPL.O,MSFT.O,INTC.O,AMZN.O
Date,,,,
2010-01-04,30.572827,30.950,20.88,133.90
2010-01-05,30.625684,30.960,20.87,134.69
2010-01-06,30.138541,30.770,20.80,132.25
2010-01-07,30.082827,30.452,20.60,130.00
2010-01-08,30.282827,30.660,20.83,133.52


In [5]:
df_stock.columns

Index(['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O'], dtype='object')

In [6]:
def BAIS(stock, window):
    stock_bais = stock
    for i in stock.columns:
        stock_bais[i+'_BAIS'] = stock[i].rolling(window).mean()
    return stock_bais


In [7]:
df_stock_bais = BAIS(df_stock, 1)

/Users/user/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
df_stock_bais.corr().values

array([[1.        , 0.94697286, 0.94499423, 0.93054298, 1.        ,
        0.94697286, 0.94499423, 0.93054298],
       [0.94697286, 1.        , 0.94396899, 0.9752662 , 0.94697286,
        1.        , 0.94396899, 0.9752662 ],
       [0.94499423, 0.94396899, 1.        , 0.93619018, 0.94499423,
        0.94396899, 1.        , 0.93619018],
       [0.93054298, 0.9752662 , 0.93619018, 1.        , 0.93054298,
        0.9752662 , 0.93619018, 1.        ],
       [1.        , 0.94697286, 0.94499423, 0.93054298, 1.        ,
        0.94697286, 0.94499423, 0.93054298],
       [0.94697286, 1.        , 0.94396899, 0.9752662 , 0.94697286,
        1.        , 0.94396899, 0.9752662 ],
       [0.94499423, 0.94396899, 1.        , 0.93619018, 0.94499423,
        0.94396899, 1.        , 0.93619018],
       [0.93054298, 0.9752662 , 0.93619018, 1.        , 0.93054298,
        0.9752662 , 0.93619018, 1.        ]])

In [21]:
def grangers_causation_matrix(data,variables,test = 'ssr_chi2test', verbose = False):
    maxlag = 1
    test = 'ssr_chi2test'
    X_train = pd.DataFrame(np.zeros((len(variables),len(variables))),columns = variables, index = variables)
    for c in X_train.columns:
        for r in X_train.index:
            test_result = grangercausalitytests(data[[r,c]],maxlag = maxlag, verbose = False)
            #print(data[[r,c]])
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            #print(p_values)
            if verbose: print(f'y={r},x={c}, P values = {p_values}')
            min_p_value = np.min(p_values)
            X_train.loc[r,c] = min_p_value
    X_train.columns = [var + '_x' for var in variables]
    X_train.index = [var + '_y' for var in variables]
    return X_train

In [22]:
def processingforcausality(data):
    causality = grangers_causation_matrix(data,data.columns,test = 'ssr_chi2test', verbose = False)
    return causality.fillna(0)

In [38]:
def patternofcausality(data):
    causality = processingforcausality(data)
    return causality

In [39]:
def graphconstruction(df,size):
    maxlag = 20
    test = 'ssr_chi2test'
    X = []
    new = patternofcausality(df.iloc[:size])
    X.append(new)
    i = 1
    while i<=df.shape[0]-size+1:
    #print(i)
        new = patternofcausality(df.iloc[i:i+size])
    #print(new)
        X.append(new)
    #From.append(df.index[pt[i]])
    #End.append(df.index[pt[i+2]])
        i=i+size
        print(i)
    return np.array(X)

In [40]:
X = graphconstruction(df_stock_bais,20)

21
41
61
81
101
121
141
161
181
201
221
241
261
281
301
321
341
361
381
401
421
441
461
481
501
521
541
561
581
601
621
641
661
681
701
721
741
761
781
801
821
841
861
881
901
921
941
961
981
1001
1021
1041
1061
1081
1101
1121
1141
1161
1181
1201
1221
1241
1261
1281
1301
1321
1341
1361
1381
1401
1421
1441
1461
1481
1501
1521
1541
1561
1581
1601
1621
1641
1661
1681
1701
1721
1741
1761
1781
1801
1821
1841
1861
1881
1901
1921
1941
1961
1981
2001
2021
2041
2061
2081
2101
2121
2141
2161
2181
2201
2221
2241
2261
2281
2301
2321
2341
2361
2381
2401
2421
2441
2461
2481
2501


In [41]:
X.shape

(126, 8, 8)

In [42]:
X

array([[[1.000e+00, 4.310e-02, 8.815e-01, ..., 4.310e-02, 8.815e-01,
         8.103e-01],
        [4.223e-01, 1.000e+00, 6.499e-01, ..., 1.000e+00, 6.499e-01,
         8.620e-01],
        [2.513e-01, 4.920e-02, 1.000e+00, ..., 4.920e-02, 1.000e+00,
         3.717e-01],
        ...,
        [4.223e-01, 1.000e+00, 6.499e-01, ..., 1.000e+00, 6.499e-01,
         8.620e-01],
        [2.513e-01, 4.920e-02, 1.000e+00, ..., 4.920e-02, 1.000e+00,
         3.717e-01],
        [7.700e-03, 1.260e-02, 2.778e-01, ..., 1.260e-02, 2.778e-01,
         1.000e+00]],

       [[1.000e+00, 4.020e-02, 7.806e-01, ..., 4.020e-02, 7.806e-01,
         9.167e-01],
        [3.884e-01, 1.000e+00, 7.256e-01, ..., 1.000e+00, 7.256e-01,
         7.421e-01],
        [3.278e-01, 7.090e-02, 1.000e+00, ..., 7.090e-02, 1.000e+00,
         4.762e-01],
        ...,
        [3.884e-01, 1.000e+00, 7.256e-01, ..., 1.000e+00, 7.256e-01,
         7.421e-01],
        [3.278e-01, 7.090e-02, 1.000e+00, ..., 7.090e-02, 1.000e+00,
   

In [43]:
np.savetxt('DynamicNetwork.csv', X.reshape(126,-1),delimiter=',')